<a href="https://colab.research.google.com/github/Ericfeng84/Leads_attribution/blob/main/Leads_Attribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime, timedelta

# 模拟销售线索数据
leads_data = {
    'lead_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'customer_id': [101, 101, 101, 102, 102, 102, 103, 103, 103, 101, 101, 104],
    'lead_timestamp': [
        '2023-01-01 10:00:00', '2023-01-15 12:00:00', '2023-03-05 14:00:00', # Cust 101
        '2023-02-01 09:00:00', '2023-02-10 11:00:00', '2023-04-10 16:00:00', # Cust 102 (lead 6 is >90 days before order)
        '2023-05-01 08:00:00', '2023-05-15 10:00:00', '2023-05-20 12:00:00', # Cust 103
        '2023-06-01 10:00:00', '2023-08-20 11:00:00', # Cust 101 (for second order)
        '2023-09-01 10:00:00'  # Cust 104 (no order)
    ],
    'lead_source': ['Email', 'Social', 'Search', 'Referral', 'Display', 'Email', 'Social', 'Search', 'Email', 'Organic', 'PPC', 'Direct']
}
leads_df = pd.DataFrame(leads_data)
leads_df['lead_timestamp'] = pd.to_datetime(leads_df['lead_timestamp'])

# 模拟订单数据
orders_data = {
    'order_id': [1001, 1002, 1003, 1004],
    'customer_id': [101, 102, 103, 101],
    'order_timestamp': [
        '2023-03-10 10:00:00', # Cust 101 Order 1
        '2023-03-15 12:00:00', # Cust 102 Order 1
        '2023-05-25 14:00:00', # Cust 103 Order 1
        '2023-08-25 18:00:00'  # Cust 101 Order 2
    ],
    'order_value': [100, 150, 200, 120]
}
orders_df = pd.DataFrame(orders_data)
orders_df['order_timestamp'] = pd.to_datetime(orders_df['order_timestamp'])

print("--- 销售线索 (Leads) ---")
print(leads_df)
print("\n--- 订单 (Orders) ---")
print(orders_df)


--- 销售线索 (Leads) ---
    lead_id  customer_id      lead_timestamp lead_source
0         1          101 2023-01-01 10:00:00       Email
1         2          101 2023-01-15 12:00:00      Social
2         3          101 2023-03-05 14:00:00      Search
3         4          102 2023-02-01 09:00:00    Referral
4         5          102 2023-02-10 11:00:00     Display
5         6          102 2023-04-10 16:00:00       Email
6         7          103 2023-05-01 08:00:00      Social
7         8          103 2023-05-15 10:00:00      Search
8         9          103 2023-05-20 12:00:00       Email
9        10          101 2023-06-01 10:00:00     Organic
10       11          101 2023-08-20 11:00:00         PPC
11       12          104 2023-09-01 10:00:00      Direct

--- 订单 (Orders) ---
   order_id  customer_id     order_timestamp  order_value
0      1001          101 2023-03-10 10:00:00          100
1      1002          102 2023-03-15 12:00:00          150
2      1003          103 2023-05-25 14:00:0

In [2]:
# 合并订单和线索数据
merged_df = pd.merge(orders_df, leads_df, on='customer_id', how='left')

# 筛选有效的线索：
# 1. 线索时间必须早于或等于订单时间
# 2. 线索时间必须在订单时间前的90天内
merged_df = merged_df[merged_df['lead_timestamp'] <= merged_df['order_timestamp']]
merged_df['time_diff_to_order'] = merged_df['order_timestamp'] - merged_df['lead_timestamp']
merged_df = merged_df[merged_df['time_diff_to_order'] <= timedelta(days=90)]

# 对于没有有效线索的订单，它们在合并后lead_id等列会是NaN，我们先保留它们，后续模型会处理
print("\n--- 合并并筛选后的有效线索-订单对 ---")
# 按订单ID和线索时间排序，方便查看
print(merged_df.sort_values(by=['order_id', 'lead_timestamp']))



--- 合并并筛选后的有效线索-订单对 ---
    order_id  customer_id     order_timestamp  order_value  lead_id  \
0       1001          101 2023-03-10 10:00:00          100        1   
1       1001          101 2023-03-10 10:00:00          100        2   
2       1001          101 2023-03-10 10:00:00          100        3   
5       1002          102 2023-03-15 12:00:00          150        4   
6       1002          102 2023-03-15 12:00:00          150        5   
8       1003          103 2023-05-25 14:00:00          200        7   
9       1003          103 2023-05-25 14:00:00          200        8   
10      1003          103 2023-05-25 14:00:00          200        9   
14      1004          101 2023-08-25 18:00:00          120       10   
15      1004          101 2023-08-25 18:00:00          120       11   

        lead_timestamp lead_source time_diff_to_order  
0  2023-01-01 10:00:00       Email   68 days 00:00:00  
1  2023-01-15 12:00:00      Social   53 days 22:00:00  
2  2023-03-05 14:00:00   

In [3]:
merged_df

,order_id,customer_id,order_timestamp,order_value,lead_id,lead_timestamp,lead_source,time_diff_to_order
0,1001,101,2023-03-10 10:00:00,100,1,2023-01-01 10:00:00,Email,68 days 00:00:00
1,1001,101,2023-03-10 10:00:00,100,2,2023-01-15 12:00:00,Social,53 days 22:00:00
2,1001,101,2023-03-10 10:00:00,100,3,2023-03-05 14:00:00,Search,4 days 20:00:00
5,1002,102,2023-03-15 12:00:00,150,4,2023-02-01 09:00:00,Referral,42 days 03:00:00
6,1002,102,2023-03-15 12:00:00,150,5,2023-02-10 11:00:00,Display,33 days 01:00:00
8,1003,103,2023-05-25 14:00:00,200,7,2023-05-01 08:00:00,Social,24 days 06:00:00
9,1003,103,2023-05-25 14:00:00,200,8,2023-05-15 10:00:00,Search,10 days 04:00:00
10,1003,103,2023-05-25 14:00:00,200,9,2023-05-20 12:00:00,Email,5 days 02:00:00
14,1004,101,2023-08-25 18:00:00,120,10,2023-06-01 10:00:00,Organic,85 days 08:00:00
15,1004,101,2023-08-25 18:00:00,120,11,2023-08-20 11:00:00,PPC,5 days 07:00:00


In [4]:
def calculate_attribution(df, model_type, time_decay_half_life_days=7, position_based_weights=(0.4, 0.2, 0.4)):
    """
    计算归因分数

    Args:
        df (pd.DataFrame): 包含 'order_id', 'lead_id', 'lead_timestamp', 'order_timestamp' 的DataFrame，
                           其中每行代表一个订单和一个有效的潜在贡献线索。
        model_type (str): 归因模型类型: 'last_touch', 'first_touch', 'linear',
                                      'time_decay', 'position_based'.
        time_decay_half_life_days (int): 时间衰减模型的半衰期（天）。
        position_based_weights (tuple): 位置模型的权重 (first, middle_sum, last)。

    Returns:
        pd.DataFrame: 包含 'order_id', 'lead_id', 'attribution_score' 的DataFrame。
    """
    if df.empty or 'lead_id' not in df.columns or df['lead_id'].isnull().all():
        print(f"警告: 对于模型 {model_type}, 没有有效的线索-订单对进行归因。")
        return pd.DataFrame(columns=['order_id', 'lead_id', 'attribution_score'])

    # 确保按订单ID和线索时间排序，这对于某些模型很重要
    df = df.sort_values(by=['order_id', 'lead_timestamp'])

    attribution_results = []

    for order_id, group in df.groupby('order_id'):
        # 如果一个订单没有任何有效的线索（例如，merged_df中该order_id对应的lead_id都是NaN）
        # 这种情况应该在传入df之前就被过滤掉了，但以防万一
        if group['lead_id'].isnull().all():
            continue

        num_touchpoints = len(group)

        if model_type == 'last_touch':
            # 最后一个触点获得100%功劳
            last_touch_lead_id = group.iloc[-1]['lead_id']
            for _, row in group.iterrows():
                score = 1.0 if row['lead_id'] == last_touch_lead_id else 0.0
                attribution_results.append({'order_id': order_id, 'lead_id': row['lead_id'], 'lead_source': row['lead_source'], 'attribution_score': score})

        elif model_type == 'first_touch':
            # 第一个触点获得100%功劳
            first_touch_lead_id = group.iloc[0]['lead_id']
            for _, row in group.iterrows():
                score = 1.0 if row['lead_id'] == first_touch_lead_id else 0.0
                attribution_results.append({'order_id': order_id, 'lead_id': row['lead_id'], 'lead_source': row['lead_source'], 'attribution_score': score})

        elif model_type == 'linear':
            # 所有触点平分功劳
            score = 1.0 / num_touchpoints if num_touchpoints > 0 else 0
            for _, row in group.iterrows():
                attribution_results.append({'order_id': order_id, 'lead_id': row['lead_id'], 'lead_source': row['lead_source'], 'attribution_score': score})

        elif model_type == 'time_decay':
            # 离转化越近的触点，功劳越大
            # 权重计算公式: 2^(-time_diff_days / half_life)
            # time_diff_days 是指转化发生前多少天发生的触点

            # 'time_diff_to_order' 是 timedelta 对象，我们需要天数
            group['days_before_conversion'] = group['time_diff_to_order'].dt.days

            # 计算每个触点的原始权重
            group['raw_weight'] = 2 ** (-group['days_before_conversion'] / time_decay_half_life_days)

            total_weight = group['raw_weight'].sum()

            if total_weight > 0:
                group['attribution_score'] = group['raw_weight'] / total_weight
            else: # 如果所有权重都是0（例如，所有触点都在转化当天，days_before_conversion=0，半衰期很小）
                  # 或者只有一个触点且权重为0（不太可能，除非半衰期为0）
                  # 这种情况下，可以平均分配，或者给最后一个（取决于业务逻辑）
                  # 这里我们选择平均分配
                group['attribution_score'] = 1.0 / num_touchpoints if num_touchpoints > 0 else 0

            for _, row in group.iterrows():
                 attribution_results.append({'order_id': order_id, 'lead_id': row['lead_id'], 'lead_source': row['lead_source'], 'attribution_score': row['attribution_score']})

        elif model_type == 'position_based':
            # U型模型：首尾触点分配较多权重，中间触点平分剩余权重
            first_weight, middle_sum_weight, last_weight = position_based_weights

            if num_touchpoints == 1:
                # 只有一个触点，获得100%
                row = group.iloc[0]
                attribution_results.append({'order_id': order_id, 'lead_id': row['lead_id'], 'lead_source': row['lead_source'], 'attribution_score': 1.0})
            elif num_touchpoints == 2:
                # 两个触点，通常平分，或者按首尾权重比例分配
                # 这里我们按首尾权重比例分配并归一化
                # 或者简单地给它们各50%
                # 为了与多触点情况下的逻辑一致，我们用首尾权重，然后归一化
                # 或者更简单：如果只有两个，第一个得到 first_weight，最后一个得到 last_weight，然后归一化
                # 假设 position_based_weights = (0.4, 0.2, 0.4)
                # 那么第一个拿0.4/(0.4+0.4) = 0.5, 第二个拿0.4/(0.4+0.4) = 0.5
                total_allocatable_weight = first_weight + last_weight

                first_row = group.iloc[0]
                attribution_results.append({'order_id': order_id, 'lead_id': first_row['lead_id'], 'lead_source': first_row['lead_source'], 'attribution_score': first_weight / total_allocatable_weight})

                last_row = group.iloc[1]
                attribution_results.append({'order_id': order_id, 'lead_id': last_row['lead_id'], 'lead_source': last_row['lead_source'], 'attribution_score': last_weight / total_allocatable_weight})

            else: # num_touchpoints > 2
                middle_touchpoints_count = num_touchpoints - 2
                weight_per_middle_touchpoint = middle_sum_weight / middle_touchpoints_count if middle_touchpoints_count > 0 else 0

                for i, (_, row) in enumerate(group.iterrows()):
                    score = 0.0
                    if i == 0: # First touch
                        score = first_weight
                    elif i == num_touchpoints - 1: # Last touch
                        score = last_weight
                    else: # Middle touches
                        score = weight_per_middle_touchpoint
                    attribution_results.append({'order_id': order_id, 'lead_id': row['lead_id'], 'lead_source': row['lead_source'], 'attribution_score': score})
        else:
            raise ValueError(f"未知的模型类型: {model_type}")

    return pd.DataFrame(attribution_results)

# 清理 merged_df，只保留有有效 lead_id 的行，这些是实际的触点
# 如果一个订单没有任何有效线索，它不应该参与归因计算
valid_touchpoints_df = merged_df.dropna(subset=['lead_id']).copy() # 使用 .copy() 避免 SettingWithCopyWarning

# 测试各个模型
print("\n--- 最终互动模型 (Last Touch) ---")
last_touch_attribution = calculate_attribution(valid_touchpoints_df, 'last_touch')
print(last_touch_attribution[last_touch_attribution['attribution_score'] > 0]) # 只显示有功劳的

print("\n--- 首次互动模型 (First Touch) ---")
first_touch_attribution = calculate_attribution(valid_touchpoints_df, 'first_touch')
print(first_touch_attribution[first_touch_attribution['attribution_score'] > 0])

print("\n--- 线性模型 (Linear) ---")
linear_attribution = calculate_attribution(valid_touchpoints_df, 'linear')
print(linear_attribution)

print("\n--- 时间衰减模型 (Time Decay, 半衰期7天) ---")
time_decay_attribution = calculate_attribution(valid_touchpoints_df, 'time_decay', time_decay_half_life_days=7)
print(time_decay_attribution)

print("\n--- 位置模型 (Position-Based / U-Shaped, 40%-20%-40%) ---")
position_based_attribution = calculate_attribution(valid_touchpoints_df, 'position_based', position_based_weights=(0.4, 0.2, 0.4))
print(position_based_attribution)



--- 最终互动模型 (Last Touch) ---
   order_id  lead_id lead_source  attribution_score
2      1001        3      Search                1.0
4      1002        5     Display                1.0
7      1003        9       Email                1.0
9      1004       11         PPC                1.0

--- 首次互动模型 (First Touch) ---
   order_id  lead_id lead_source  attribution_score
0      1001        1       Email                1.0
3      1002        4    Referral                1.0
5      1003        7      Social                1.0
8      1004       10     Organic                1.0

--- 线性模型 (Linear) ---
   order_id  lead_id lead_source  attribution_score
0      1001        1       Email           0.333333
1      1001        2      Social           0.333333
2      1001        3      Search           0.333333
3      1002        4    Referral           0.500000
4      1002        5     Display           0.500000
5      1003        7      Social           0.333333
6      1003        8      Search  